<a href="https://colab.research.google.com/github/PasanAbeysekara/DataScience/blob/main/LoanPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd 

In [ ]:
import os
for dirname, _, filenames in os.walk('/content/LoanData'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

Data

In [ ]:
df = pd.read_csv("/content/LoadData/loan.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (47) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df["loan_status"].replace("Fully Paid",1,inplace=True)
df["loan_status"].replace("Current",1,inplace=True)
df["loan_status"].replace("Charged Off",0,inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2)

remove null

In [ ]:
null_cols = []
for col in df.columns:
  print(col,df[col].isna().sum())
  if(df[col].isna().sum()>1000):
    null_cols.append(col)
  
df = df.drop(null_cols,axis=1)

df.dropna(inplace=True)

id 0
member_id 0
loan_amnt 0
funded_amnt 0
funded_amnt_inv 0
term 0
int_rate 0
installment 0
grade 0
sub_grade 0
emp_title 2459
emp_length 1075
home_ownership 0
annual_inc 0
verification_status 0
issue_d 0
loan_status 0
pymnt_plan 0
url 0
desc 12940
purpose 0
title 11
zip_code 0
addr_state 0
dti 0
delinq_2yrs 0
earliest_cr_line 0
inq_last_6mths 0
mths_since_last_delinq 25682
mths_since_last_record 36931
open_acc 0
pub_rec 0
revol_bal 0
revol_util 50
total_acc 0
initial_list_status 0
out_prncp 0
out_prncp_inv 0
total_pymnt 0
total_pymnt_inv 0
total_rec_prncp 0
total_rec_int 0
total_rec_late_fee 0
recoveries 0
collection_recovery_fee 0
last_pymnt_d 71
last_pymnt_amnt 0
next_pymnt_d 38577
last_credit_pull_d 2
collections_12_mths_ex_med 56
mths_since_last_major_derog 39717
policy_code 0
application_type 0
annual_inc_joint 39717
dti_joint 39717
verification_status_joint 39717
acc_now_delinq 0
tot_coll_amt 39717
tot_cur_bal 39717
open_acc_6m 39717
open_il_6m 39717
open_il_12m 39717
open_il_2

data continue

In [ ]:
# df_train = pd.read_csv("../content/titanic/train.csv")

# df_test = pd.read_csv('../content/titanic/test.csv')

y = df_train['loan_status']

df_train.drop(['loan_status','id','member_id'],axis=1,inplace=True)

test_index = df_test['id']
df_test.drop(['id','member_id'],axis=1,inplace=True)

Define Preprocessing steps

In [ ]:
numerical_cols= df_train.select_dtypes(exclude='object').columns

catagorical_cols = df_train.select_dtypes(include='object').columns

numerical_transfer = SimpleImputer(strategy='median')

catagorical_transfer = Pipeline(steps=
                                [
                                    ('imputer',SimpleImputer(strategy='most_frequent')),
                                    ('onehot',OneHotEncoder(handle_unknown='ignore'))
                                ])

preprocessor = ColumnTransformer(transformers=
                                 [
                                     ('num',numerical_transfer,numerical_cols),
                                     ('cat',catagorical_transfer,catagorical_cols)
                                 ])

Define the model

In [ ]:
from xgboost import XGBRegressor

In [ ]:
model = XGBRegressor(n_estimators=1000,random_state=0,learning_rate=0.05)

my_pipeline =Pipeline(steps = 
                     [
                         ('preprocessor',preprocessor),
                         ('model',model)
                     ])

Submission

In [ ]:
my_pipeline.fit(df_train,y)

pred = my_pipeline.predict(df_test)

output = pd.DataFrame({'id':test_index,
                      'loan_status':pred})

for id in range(len(output['id'])):
  if(output['loan_status'][id]>0.5):
    output['loan_status'][id]=1
  else:
    output['loan_status'][id]=0

output.to_csv('xgboost_loan-pred_1.csv',index=False)

pd.read_csv('xgboost_loan-pred_1.csv')

In [ ]:
df_test.loan_status

32868    0
10387    1
32431    1
25256    1
38487    1
        ..
5333     1
27886    1
6811     0
7942     1
911      0
Name: loan_status, Length: 7944, dtype: int64

In [ ]:
output = pd.read_csv('/content/xgboost_loan-pred_1.csv')

In [ ]:
output.loan_status

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
7939    0.0
7940    0.0
7941    1.0
7942    1.0
7943    1.0
Name: loan_status, Length: 7944, dtype: float64

In [ ]:
mergedStuff = pd.merge(df_test, output, on=['loan_status'], how='inner')
mergedStuff.head()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np

In [ ]:
df_ = pd.DataFrame([[2, 2], [3, 3]], columns = ["col1", "col2"])

comparison_column = np.where(df_["col1"] == df_["col2"], True, False)

In [ ]:
comparison_column.sum()

2